# Svenskt Kvinnobiografiskt lexikon part 3
version part 3 - 1.0


* this [Jupityr Notebook](https://github.com/salgo60/open-data-examples/blob/master/Svenskt%20Kvinnobiografiskt%20lexikon%20part%203.ipynb) 
  * [part 1](https://github.com/salgo60/open-data-examples/blob/master/Svenskt%20Kvinnobiografiskt%20lexikon.ipynb) med sökfrågor etc.
  * [part 2](https://github.com/salgo60/open-data-examples/blob/master/Svenskt%20Kvinnobiografiskt%20lexikon%20part%202.ipynb) med sökfrågor etc.
  

# Get metadata SKBL  

# Wikidata

In [4]:
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

import sys
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

query = """SELECT (REPLACE(STR(?item), ".*Q", "Q") AS ?wid)  ?skblid WHERE {
?item wdt:P31 wd:Q5.
?item wdt:P4963 ?skblid
} order by ?skbl"""

def get_sparql_dataframe(endpoint_url, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    user_agent = "salgo60/%s.%s" % (sys.version_info[0], sys.version_info[1])
 
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

WDtot = get_sparql_dataframe(endpoint_url, query)
 
WDtot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 2 columns):
wid       1460 non-null object
skblid    1460 non-null object
dtypes: object(2)
memory usage: 22.9+ KB


In [5]:
WDtot.sort_index(inplace=True)  

In [6]:
WDtot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 2 columns):
wid       1460 non-null object
skblid    1460 non-null object
dtypes: object(2)
memory usage: 22.9+ KB


In [39]:
import csv  
import urllib3, json
http = urllib3.PoolManager() 

jsonURL = "https://skbl.se/sv/artikel/"
listNewItems =[]
for index,row in WDtot.iterrows():
    i = i+1
    url = jsonURL + row["skblid"] + ".json"
    r = http.request('GET', url) 
    #print(i, url)
    data = json.loads(r.data.decode('utf-8'))  
    new_item = dict()
    new_item['skblid'] = row["skblid"]
    new_item['wid'] = row["wid"]
    try:
        publishedDate = data['publication_date']        
    except:
        publishedDate =""
    new_item['publishedDate'] = publishedDate
    try:
        article_author_firstname = data['article_author'][0]['firstname']
    except:
        article_author_firstname =""
    new_item['article_author_firstname'] = article_author_firstname
    try:
        article_author_lastname = data['article_author'][0]['lastname']
    except:
        article_author_lastname =""
    new_item['article_author_lastname'] = article_author_lastname
    try:
        subtitle_sv = data['subtitle']
    except:
        subtitle_sv =""
    new_item['subtitle_sv'] = subtitle_sv
    try:
        subtitle_eng = data['subtitle_eng']
    except:
        subtitle_eng =""
    try:
        name_lastname = data['name']['lastname']
    except:
        name_lastname = ""
    new_item['name_lastname'] = name_lastname
    try:
        name_firstname = data['name']['firstname']
    except:
        name_firstname = ""
    new_item['name_firstname'] = name_firstname
    listNewItems.append(new_item)
print (len(listNewItems) ," antal poster")


1460  antal poster


In [40]:
if len(listNewItems) > 0:
    keys = listNewItems[0].keys()
    with open("SKBLAuthor_newfile.csv", "w", newline='') as SKBLfile:
        dict_writer = csv.DictWriter(SKBLfile, keys)
        dict_writer.writeheader()
        dict_writer.writerows(listNewItems)

    dfSKBLAuthorcsv = pd.read_csv("SKBLAuthor_newfile.csv", sep=",")   
    
else:
    print ("Ingen fil skapas inga nya poster") 
            


In [41]:
pd.set_option('display.max_rows', None) 
dfSKBLAuthorcsv

,skblid,wid,publishedDate,article_author_firstname,article_author_lastname,subtitle_sv,name_lastname,name_firstname
0,UlrikaEleonoradydrottning,Q52933,2018-03-08,Sari,Nauman,Drottning av Sverige,NaN,"/Ulrika Eleonora d.y./, drottning"
1,AstridLindgren,Q55767,2018-03-08,Boel,Westin,"Författare, samhällsdebattör",Lindgren,/Astrid/ Anna Emilia
2,GunillaBielke,Q235659,2018-03-08,Karin,Tegenborg Falkdalen,Drottning av Sverige,NaN,"/Gunilla/, drottning"
3,KarinMansdotterdrottning,Q237937,2018-03-08,Katarina,Harrison Lindbergh,Drottning av Sverige,NaN,"/Karin Månsdotter/, drottning"
4,VivecaLindfors,Q267217,2018-03-08,Tytti,Soila,"Skådespelare, dramatiker, regissör",Lindfors,Elsa /Viveca/ Torstensdotter
5,ChristinaNilsson,Q269812,2018-03-08,Ingegerd,Björklund,Operasångare,Nilsson,/Christina/
6,AnneWibble,Q273408,2018-03-08,Anders,Johnson,"Nationalekonom, politiker, minister",Wibble,/Anne/ Marie
7,EmilieHogqvist,Q274554,2018-03-08,Hélène,Ohlsson,"Skådespelare, kunglig mätress",Högqvist,Sophie /Emilie/
8,JuliaNybergEuphrosyne,Q441950,2018-03-08,Gunilla,Hermansson,Författare,Nyberg (Euphrosyne),Christina Juliana (/Julia/)
9,EvaBonnier,Q454670,2018-03-08,Margareta,Gynning,"Konstnär, mecenat",Bonnier,/Eva/ Fredrika
